# BULK

In [79]:
import ase
from ase.db import connect
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.eos import calculate_eos

In [80]:
atom = bulk('Al', 'fcc')
atom.get_volume()

16.60753125

In [81]:
atom = bulk('Cu', 'fcc')
atom.calc =EMT()
atom.get_volume(), atom.get_potential_energy() # 想要获得能量就得指定calc方法

(11.76147025, -0.005681511358588409)

In [82]:
eos = calculate_eos(atom)
v,e,B = eos.fit() # 得到energy最小值 -- 原子的相对位置不变
v,e,B

(11.565374510214, -0.007036478731361484, 0.8392875566841875)

In [83]:
atom.cell *= (v / atom.get_volume()) ** (1/3)
atom.get_potential_energy()

-0.0070364920483623195

In [84]:
# ase.io.write("POSCAR",atom,format="vasp")

In [85]:
db = ase.db.connect('bulk.db')
for atom in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(atom, 'fcc')
    atoms.calc = EMT()
    eos = ase.eos.calculate_eos(atoms)
    v,e,B = eos.fit()
    atoms.cell *= (v/atoms.get_volume()) ** (1/3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B) # bm: bulk modulus

# ADSORBATES

## 引入1 - fcc111

In [87]:
from ase.build import fcc111
slab = ase.build.fcc111("Al", size=(2,2,3), vacuum=10, periodic=False)
ase.build.add_adsorbate(slab, 'H', 1.5, 'ontop')
slab.center(vacuum=10.0, axis=2)
# ase.io.write("POSCAR1", slab, format='vasp')
print(slab.get_tags())
len(slab)

[3 3 3 3 2 2 2 2 1 1 1 1 0]


13

## 引入2 - BFGS
BFGS 只优化原子位置，不优化胞

In [88]:
from ase.optimize import BFGS
from ase.constraints import FixAtoms

In [89]:
h2o = ase.Atoms("H2O", positions=[(0,0,0), (1,0,0), (0,1,0)], calculator=EMT())
bfgs = BFGS(h2o, trajectory="h2o.traj", restart="qn.pckl")
atoms = bfgs.run(fmax=0.1, steps=100) # fmax: 收敛精度

      Step     Time          Energy         fmax
BFGS:    0 21:41:18        2.727814        3.7034
BFGS:    1 21:41:18        2.576465        1.9057
BFGS:    2 21:41:18        2.471500        2.1184
BFGS:    3 21:41:18        2.083094        3.1839
BFGS:    4 21:41:18        2.072093        1.5143
BFGS:    5 21:41:18        1.980273        0.6710
BFGS:    6 21:41:18        1.954224        1.0407
BFGS:    7 21:41:18        1.937640        0.7327
BFGS:    8 21:41:18        1.898400        0.2292
BFGS:    9 21:41:18        1.888954        0.2511
BFGS:   10 21:41:18        1.882341        0.1968
BFGS:   11 21:41:18        1.880049        0.1114
BFGS:   12 21:41:18        1.879194        0.0516


In [90]:
symb="Al"
ads="H"
n=3 # z轴上选取三层
atoms = fcc111(symb, (1, 1, n))
ase.build.add_adsorbate(atoms, ads, height=1.0, position='fcc')
# print(atoms.get_tags()) # [3,2,1,0]
# Constrain all atoms except the adsorbate:
# print(len(atoms)) # 4
fixed = list(range(len(atoms) - 1))
# print(fixed) # [0,1,2]
# print(FixAtoms(indices=fixed))
atoms.constraints = FixAtoms(indices=fixed)
atoms.calc = EMT()
print(atoms.get_cell())
opt = BFGS(atoms, logfile="out.log")
opt.run(fmax=0.01)
print(atoms.get_cell())

Cell([[2.8637824638055176, 0.0, 0.0], [1.4318912319027588, 2.4801083645679673, 0.0], [0.0, 0.0, 0.0]])
Cell([[2.8637824638055176, 0.0, 0.0], [1.4318912319027588, 2.4801083645679673, 0.0], [0.0, 0.0, 0.0]])
